<a href="https://colab.research.google.com/github/eunShim/BigDataCapstone/blob/main/bigdatacapstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
import random
import pandas as pd
import matplotlib.pyplot as plt

# =======================================
# 1. Load KT4 sample data (500 users)
# =======================================
zip_path = "/content/EdNet-KT4.zip"
zf = zipfile.ZipFile(zip_path)


file_list = [f for f in zf.namelist() if f.endswith(".csv")]
print("총 사용자 수:", len(file_list))

# --------------------------
# 2. 무작위로 500명 샘플링
# --------------------------
sample_files = random.sample(file_list, 500)

user_stats = []

for f in sample_files:
    with zf.open(f) as fp:
        df = pd.read_csv(fp)

        # 빈 파일 처리
        if df.empty:
            continue

        # user_id 추출
        user_id = f.split("/")[-1].replace(".csv","")

        # timestamp 정렬 및 소요시간 계산
        if "timestamp" in df.columns:
            df = df.sort_values("timestamp")
            duration = df["timestamp"].iloc[-1] - df["timestamp"].iloc[0]
        else:
            duration = np.nan

        # quit 횟수
        quit_count = (df["action_type"] == "quit").sum()

        # 플랫폼 비율
        if "platform" in df.columns:
            total_actions = len(df)
            mobile_ratio = (df["platform"] == "mobile").sum() / total_actions
            web_ratio = (df["platform"] == "web").sum() / total_actions
        else:
            mobile_ratio = np.nan
            web_ratio = np.nan

        # 문제 수 대비 submit 비율
        if "action_type" in df.columns and "item_id" in df.columns:
            submit_count = (df["action_type"] == "submit").sum()
            unique_questions = df[df["item_id"].astype(str).str.startswith("q")]["item_id"].nunique()
            submit_ratio = submit_count / unique_questions if unique_questions > 0 else np.nan
        else:
            submit_count = np.nan
            submit_ratio = np.nan

        # 사용자 단위 통계 저장
        user_stats.append({
            "user_id": user_id,
            "quit_count": quit_count,
            "duration_ms": duration,
            "mobile_ratio": mobile_ratio,
            "web_ratio": web_ratio,
            "submit_count": submit_count,
            "unique_questions": unique_questions,
            "submit_ratio": submit_ratio
        })

# --------------------------
# 3. 결과 요약
# --------------------------
stats_df = pd.DataFrame(user_stats)
print("\n✅ 사용자별 통계 미리보기:")
print(stats_df.head())

# 전체 통계 요약
print("\n📈 전체 사용자 통계 요약:")
print(stats_df.describe())

# --------------------------
# 4. 추가 시각화 (선택)
# --------------------------
import matplotlib.pyplot as plt

plt.figure(figsize=(8,4))
stats_df["quit_count"].hist(bins=30)
plt.title("Quit 행동 횟수 분포")
plt.xlabel("quit_count")
plt.ylabel("사용자 수")
plt.show()

plt.figure(figsize=(8,4))
stats_df["submit_ratio"].hist(bins=30)
plt.title("문항 수 대비 Submit 비율 분포")
plt.xlabel("submit_ratio")
plt.ylabel("사용자 수")
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/content/EdNet-KT4.zip'

집단 평균 Action Type Distribution 그래프(500명 샘플) 만 놓고, 자기효능감(Self-Efficacy)과 좌절(Frustration) 관점에서 세밀하게 분석해드리겠습니다.

🔎 1. 상위 3대 행동: enter – respond – submit (집단 비율 ≈ 78%)

enter (≈ 28%)

학습 세션에 꾸준히 진입한다는 것은 시도하려는 동기와 자기효능감이 유지되고 있음을 시사합니다.

반복적 진입은 “나는 문제를 풀 수 있다”는 기대감과 과제 수행 의지를 반영.

respond (≈ 27%) + submit (≈ 23%)

집단 전체가 단순 응답(respond)뿐 아니라 제출(submit)까지 이어간다는 점은 과제에 대한 책임감과 자기효능감을 나타냄.

응답과 제출 비율이 비슷하다는 것은 → 답안을 시도하면 끝까지 제출하는 경향이 강하다는 의미.

“시도 → 완수”로 이어지는 패턴은 높은 자기효능감의 전형적 지표.

🔎 2. quit (≈ 12%)

비교적 높은 비중의 quit 행동은 좌절감(Frustration) 의 주요 신호.

좌절의 원인일 수 있는 요인:

문제 난이도가 자기 능력을 초과한다고 느낄 때 → 자기효능감 하락

연속된 오답이나 이해 부족 → 실패 경험 축적

따라서 집단 수준에서 quit이 평균 10% 이상 나온다는 건, 학습 시스템이 중도 포기 유발 요소를 갖고 있다는 경고.

🔎 3. 멀티미디어 행동 (pause_audio, play_audio ≈ 9%)

오디오 활용도가 일정 수준 존재하는 건, 일부 학습자가 보조 학습 전략을 쓰고 있음을 의미.

그러나 평균치가 낮은 건 → 대다수 학습자에게는 멀티미디어 자원이 효능감 향상에 크게 기여하지 못하는 상태.

오히려 오디오 이용 후 quit으로 이어지는 경우가 있다면, 학습 보조 도구가 좌절 완화에 실패했을 가능성도 있음.

🔎 4. Erase / Undo Erase (≈ 4–5%)

답안을 지우고 수정하는 행동은 학습자가 자신의 응답을 메타인지적으로 점검하고 있다는 증거.

이는 긍정적일 수 있음: "내가 틀렸을 수 있으니 다시 시도해보자" → 자기효능감의 발현.

그러나 지나치게 빈번하면 → 확신 부족, 불안과 좌절감의 징후.

집단 평균에서 4–5%는 적당한 수준이므로, 대부분은 정상적인 자기 점검 행동으로 보임.

🔎 5. play_video, pause_video, pay (거의 0~2%)

영상 학습이나 결제 행위는 집단 전체에서 거의 활용되지 않음.

영상/결제가 학습 성과와 연결되지 않으니 → 학습자가 자기효능감을 얻는 경로가 문제 풀이 활동에만 편중되어 있음.

즉, 시스템 설계가 효능감 → 문제풀이, 좌절 → quit 구조로 단순화되어 있다는 신호.

✅ 종합 해석 (Self-Efficacy vs Frustration)

자기효능감 (긍정 신호)

enter–respond–submit 사이클이 압도적으로 높음 → 집단이 “과제를 풀고 완수할 수 있다”는 믿음을 지속적으로 행동으로 보여줌.

Erase/Undo Erase 비율은 적절 → 학습자들이 불확실성을 관리하면서도 결국 과제에 도전.

좌절 (부정 신호)

quit이 평균 12%는 높음 → 집단 일부는 실패 경험을 반복하며 포기로 이어짐.

멀티미디어 활용이 효능감 강화로 이어지지 못함 → 지원 리소스가 좌절 완화에 충분히 작동하지 않음.

낮은 video/pay 비율은 학습 경험 다양성이 부족 → 효능감을 획득할 수 있는 경로가 제한적.